In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [ ]:
import art
from dotenv import load_dotenv
import openai

load_dotenv()


api = art.UnslothAPI(in_process=False, wandb_project="agent-reinforcement-training")
model = await api.get_or_create_model(
    name="yes-or-no-unsloth-001",
    base_model="unsloth/Qwen2.5-14B-Instruct",
)


async def rollout(client: openai.AsyncOpenAI, prompt: str) -> art.Trajectory:
    messages: art.Messages = [
        {
            "role": "user",
            "content": prompt,
        }
    ]
    chat_completion = await client.chat.completions.create(
        messages=messages, model=model.name, max_tokens=100
    )
    choice = chat_completion.choices[0]
    content = choice.message.content
    assert isinstance(content, str)
    if content == "yes":
        reward = 0.5
    elif content == "no":
        reward = 0.75
    elif content == "maybe":
        reward = 1.0
    else:
        reward = 0.0
    return art.Trajectory(messages_and_choices=[*messages, choice], reward=reward)


async with model.openai_client(
    estimated_completion_tokens=16, verbosity=2
) as openai_client:
    for i in range(await model.get_iteration(), 1_000):
        train_groups = await art.gather_trajectories(
            (
                (rollout(openai_client, prompt) for _ in range(32))
                for prompt in [
                    f"{prefix} with {', '.join([f"'{w}'" if use_quotes else w for w in words]) if len(words) == 3 else f'{words[0]}' + (f' or {words[1]}' if len(words) > 1 else '')}"
                    for prefix in ["respond", "just respond"]
                    for use_quotes in [True, False]
                    for words in [
                        ["yes", "no", "maybe"],
                        ["maybe", "yes", "no"],
                        ["no", "yes", "maybe"],
                        ["yes", "maybe", "no"],
                        ["yes", "no"],
                        ["maybe", "no"],
                        ["no", "maybe"],
                        ["no", "yes"],
                        ["yes", "no"],
                    ]
                ]
            ),
            pbar_desc="train",
            stream_chat_completions=8,
        )
        await model.tune(
            train_groups,
            config=art.TuneConfig(
                lr=1e-4, sequence_length=8192, plot_tensors=False, verbosity=2
            ),
        )